In [1]:
import os
import json
import time
import pandas as pd
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from webdrivermanager import GeckoDriverManager

In [2]:
def reformat(key, outdir):
    while os.path.isfile(outdir + "data.csv") == False:
        time.sleep(1)
                         
    reader = open(outdir + "data.csv","r",encoding="iso-8859-2")    
    content = reader.read().split("\n")
    if(len(content) > 0):
        writer = open(outdir + key + ".csv","w",encoding="utf8")
        # print(len(content[17:])
        for c in content[17:]:
            writer.write(c+"\n")
        writer.close()        
    reader.close()
    
    return len(content)

In [29]:
def create_browser():
    profile = webdriver.FirefoxProfile()
    profile.set_preference('browser.download.folderList', 2) # custom location
    profile.set_preference('browser.download.manager.showWhenStarting', False)
    profile.set_preference('browser.download.dir', outdir)
    profile.set_preference('browser.helperApps.neverAsk.saveToDisk', 'text/csv')
    options = Options()
    options.headless = True
    return webdriver.Firefox(profile, options=options, executable_path=geckodriver)

In [3]:
with open("data/selenium/data.json","r",encoding="utf8") as jf:
    stocks = json.load(jf)

In [18]:
outdir = '/home/lakrob/work/tmp/selenium/'
geckodriver=r'/home/lakrob/.local/share/WebDriverManager/gecko/v0.32.0/geckodriver-v0.32.0-linux64/geckodriver'

In [21]:
list(stocks.keys())[:5]

['HU0000720271',
 'HU0000706361',
 'HU0000728290',
 'HU0000715891',
 'HU0000716451']

In [34]:
while len(list(stocks.keys())) > 0:    
    errors = {}
    browser = create_browser()
    pbar = tqdm(stocks.keys())
    for key in pbar:
        # print(key)
        # key = "HU0000716451"
        while os.path.isfile(outdir + "data(1).csv") == True:
            os.remove(outdir + "data(1).csv")
            time.sleep(1)

        pbar.set_description("Downloading: " + key)
        browser.get("https://www.bamosz.hu/alapoldal?isin=" + key)
        button = browser.find_element("id", "A3225:j_idt8:j_idt273")
        click = button.click()
        time.sleep(1)

        pbar.set_description("Reformat data.csv")
        lenc = reformat(key, outdir)

        if lenc < 10:
            errors[key] = stocks[key]
            pbar.set_description("Empty: " + key)
            time.sleep(1)
            browser = create_browser()
        time.sleep(1)

        pbar.set_description("Removing data.csv")
        while os.path.isfile(outdir + "data.csv") == True:
            os.remove(outdir + "data.csv")
            time.sleep(1)

        pbar.set_description("Completed: " + key + ", len(" + str(lenc) + ")")        
        time.sleep(1)
        
    stocks = errors            
    time.sleep(1)

/tmp/ipykernel_1656/381067596.py:2: DeprecationWarning: firefox_profile has been deprecated, please use an Options object
  profile = webdriver.FirefoxProfile()
/tmp/ipykernel_1656/381067596.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  return webdriver.Firefox(profile, options=options, executable_path=geckodriver)
/tmp/ipykernel_1656/381067596.py:9: DeprecationWarning: firefox_profile has been deprecated, please pass in an Options object
  return webdriver.Firefox(profile, options=options, executable_path=geckodriver)
Completed: HU0000706247, len(3005): 100%|██████████| 2/2 [00:14<00:00,  7.18s/it]


In [35]:
stocks = errors
stocks

{}